In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
!ls ../input/hm-public-submissions

# Predictions in this competition are a list of 12 itens ordered by most relevant first.
# In this notebook I will show how to ensemble lists of different models
# To ensemble I used submissions from 3 public notebooks:
- LB: 0.0225 - https://www.kaggle.com/lichtlab/0-0226-byfone-chris-combination-approach/data?scriptVersionId=89289696
- LB: 0.0225 - https://www.kaggle.com/lunapandachan/h-m-trending-products-weekly-add-test/notebook
- LB: 0.0217 - https://www.kaggle.com/tarique7/hnm-exponential-decay-with-alternate-items/notebook

In [ ]:
sub0 = pd.read_csv('../input/hm-public-submissions/0-0226-byfone-chris-combination-approach.csv').sort_values('customer_id').reset_index(drop=True)
sub1 = pd.read_csv('../input/hm-public-submissions/h-m-trending-products-weekly-add-test.csv').sort_values('customer_id').reset_index(drop=True)
sub2 = pd.read_csv('../input/hm-public-submissions/hnm-exponential-decay-with-alternate-items.csv').sort_values('customer_id').reset_index(drop=True)

sub0.shape, sub1.shape, sub2.shape

In [ ]:
# How many predictions are in common between models

print((sub0['prediction']==sub1['prediction']).mean())
print((sub0['prediction']==sub2['prediction']).mean())
print((sub1['prediction']==sub2['prediction']).mean())

In [ ]:
print( sub0.head() )
print()
print( sub1.head() )
print()
print( sub2.head() )

In [ ]:
sub0.columns = ['customer_id', 'prediction0']
sub0['prediction1'] = sub1['prediction']
sub0['prediction2'] = sub2['prediction']
del sub1, sub2
sub0.head()

In [ ]:
def cust_blend(dt, W = [1,1,1]):
    #Global ensemble weights
    #W = [1.15,0.95,0.85]
    
    #Create a list of all model predictions
    REC = []
    REC.append(dt['prediction0'].split())
    REC.append(dt['prediction1'].split())
    REC.append(dt['prediction2'].split())
    
    #Create a dictionary of items recommended. 
    #Assign a weight according the order of appearance and multiply by global weights
    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))
    
    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())
    
    # Return the top 12 itens only
    return ' '.join(res[:12])

sub0['prediction'] = sub0.apply(cust_blend, W = [1.05,1.00,0.95], axis=1)
sub0.head()

In [ ]:
# How many predictions are in common with ensemble

print((sub0['prediction']==sub0['prediction0']).mean())
print((sub0['prediction']==sub0['prediction1']).mean())
print((sub0['prediction']==sub0['prediction2']).mean())

# Make a submission

In [ ]:
del sub0['prediction0']
del sub0['prediction1']
del sub0['prediction2']
sub0.to_csv('submission-blend-1.csv', index=False)